In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
from scipy import stats 
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import os
import time
import gc
import pylab
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor,Lasso,LinearRegression,Ridge
from sklearn.svm import SVR
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
import random
import joblib

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
data_train = pd.read_csv('/content/train.csv')
data_test = pd.read_csv('/content/test.csv')
#selecting random quary point from train set(with loss variable) and test set(without loss variable)
rand_data1 = random.choice(data_train.values)
rand_data2 = random.choice(data_test.values)

#creating dataframe for two query point
data_point1 = pd.DataFrame(data=rand_data1.reshape(1,-1),columns=data_train.columns)
data_point2 = pd.DataFrame(data=rand_data2.reshape(1,-1),columns=data_test.columns)

data_point1

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,147361,B,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,...,A,C,D,D,E,C,A,P,H,A,A,A,F,G,G,G,D,BI,EG,E,E,AN,A,K,GS,0.304143,0.422197,0.61366,0.189137,0.281143,0.379574,0.622473,0.28768,0.35897,0.26029,0.511698,0.519456,0.271571,0.806096,2825.85


In [5]:
data_point2

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,189635,B,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,G,H,D,B,C,E,E,D,T,I,C,A,A,H,E,I,Q,K,BI,CO,A,AG,AJ,A,P,HX,0.099045,0.785784,0.215772,0.432728,0.397069,0.516811,0.434643,0.29758,0.19862,0.56081,0.461077,0.450922,0.743962,0.78722


In [6]:
#X1 when we have actual 'loss' value for evaluation metric
X1 = data_point1.drop(['id','loss'],axis = 1)
y1 = data_point1['loss']

#X2  when we not have dependent variable 'loss'
X2 = data_point2.drop(['id'],axis = 1)

In [ ]:
'''
#loading essential variables and best model(Xgboost) that we found
loaded_model = joblib.load('/content/finalized_model1.sav')
label_encoded_map_dict = np.load('/content/label_encoded_map_dict.npy',allow_pickle='TRUE').item()
feature_level_count_train_dict = np.load('/content/feature_level_count_train_dict.npy',allow_pickle='TRUE').item()

categorical_features_train = np.load('/content/categorical_features_train.npy',allow_pickle='TRUE')
'''

In [12]:
#preprocessing query point
#In this function we will put 'NaN' if any level in categorical feature is missing in query point.

def search_feature(x,combine_set):
  #print(combine_set)
  if x in combine_set:
      return np.nan
  return x

def preprocessing(X,feature_level_count_train_dict,categorical_features_train):

 
  X_test =  X.copy()
  for col in categorical_features_train:
    if (set(feature_level_count_train_dict[col].keys()) - set(X_test[col].unique())) or (set(X_test[col].unique()) - set(feature_level_count_train_dict[col].keys())) :
      #print('yes')
      unique_train = set(feature_level_count_train_dict[col].keys())
      unique_test = set(X_test[col].unique())
      rem_train = unique_train-unique_test
      rem_test = unique_test-unique_train
      #print('rem_train',rem_train)
      #print('rem_test',rem_test)
      combine_set = rem_train.union(rem_test)
      #print('combine_set',combine_set)
      mapp_nan = {i: np.nan for i in combine_set }
      #print( mapp_nan)
      #uniq_nan_testdf['cat92'] = uniq_nan_testdf['cat92'].map(mapp_nan)

      X_test[col] = X_test[col].apply(lambda x: search_feature(x,combine_set),1)

  return X_test

In [13]:
#encoding of query point:
def feature_engineering(X_q,label_encoded_map_dict,categorical_features_train):
  X_test_lb_encoded = X_q.copy()

  for col in categorical_features_train:
    X_test_lb_encoded [col] = X_q[col].map(label_encoded_map_dict[col])
  return X_test_lb_encoded

### Final function with y variable

In [21]:
#combining all function to predict query point with y:
def final_func_with_y(X,y):
  #first we load variables
  categorical_features_train = np.load('/content/categorical_features_train.npy',allow_pickle='TRUE')
  feature_level_count_train_dict = np.load('/content/feature_level_count_train_dict.npy',allow_pickle='TRUE').item()
  #preprocessing Query point
  X_q = preprocessing(X,feature_level_count_train_dict,categorical_features_train)
  label_encoded_map_dict = np.load('/content/label_encoded_map_dict.npy',allow_pickle='TRUE').item()
  #encoding query point
  X_q = feature_engineering(X_q,label_encoded_map_dict,categorical_features_train).astype('float')
  #loading model
  loaded_model = joblib.load('/content/finalized_model1.sav')
  #prediction
  y_pred = loaded_model.predict(X_q)
  y_pred = np.exp(y_pred) - 200
  # MAE evaluation
  mae = mean_absolute_error(y_pred,y)
  
  print('prediction : ',y_pred[0])
  print('MAE score: ',mae)
  



In [22]:
start = time.time()


final_func_with_y(X1,y1)

end = time.time()
print('Total execution time for a query point is : {} sec '.format((end - start)))

prediction :  2698.0632
MAE score:  127.78676757812491
Total execution time for a query point is : 0.5385403633117676 sec 


### Final function without y variable

In [23]:
#combining all function to predict query point without y:
def final_func_without_y(X):
  #first we load variables
  categorical_features_train = np.load('/content/categorical_features_train.npy',allow_pickle='TRUE')
  feature_level_count_train_dict = np.load('/content/feature_level_count_train_dict.npy',allow_pickle='TRUE').item()
  #preprocessing Query point
  X_q = preprocessing(X,feature_level_count_train_dict,categorical_features_train)
  label_encoded_map_dict = np.load('/content/label_encoded_map_dict.npy',allow_pickle='TRUE').item()
  #encoding query point
  X_q = feature_engineering(X_q,label_encoded_map_dict,categorical_features_train).astype('float')
  #loading model
  loaded_model = joblib.load('/content/finalized_model1.sav')
  #prediction
  y_pred = loaded_model.predict(X_q)
  y_pred = np.exp(y_pred) - 200
  
  print('prediction : ',y_pred[0])
  

In [24]:
start = time.time()


final_func_without_y(X2)

end = time.time()
print('total execution time for a query point is : ',(end - start))

prediction :  993.9398
total execution time for a query point is :  0.5183806419372559
